In [39]:
import numpy as np
import pandas as pd

In [40]:
#dataset
from sklearn.datasets import load_iris,fetch_openml

In [41]:
#transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler,RobustScaler

In [42]:
#estimator
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC

In [43]:
#pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

In [44]:
from sklearn.model_selection import train_test_split#データセット分割

In [45]:
#パラメータ調整
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,ParameterGrid

In [46]:
#評価関数
from sklearn.metrics import accuracy_score,f1_score

In [47]:
X,y = load_iris(as_frame=True,return_X_y=True)

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [62]:
#パイプライン構築
##特徴量
features = ['sepal length (cm)',
            'sepal width (cm)',   
            'petal length (cm)', 
            'petal width (cm)']
##変換器パイプライン
###パイプライン定義
transformer = Pipeline(steps=[
    ('scaler',StandardScaler()) #特徴量を標準化
])
###統合（特徴量を変換器にかける）
preprocesser = ColumnTransformer(transformers=[
    ('transform',transformer,features)
])
##パイプライン全体：変換器パイプラインから予測器
pipeline = Pipeline(steps=[
    ('preprocesser',preprocesser),#変換器パイプライン
    ('classifier',LogisticRegression())#予測器（推定器）
])

In [50]:
#パイプライン表示
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', LogisticRegression())])

In [51]:
set_config(display='None')

In [52]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)

In [53]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', LogisticRegression())])

In [54]:
y_test_pred = pipeline.predict(X_test)
accuracy_score(y_test,y_test_pred)

0.9333333333333333

In [55]:
#グリッドサーチの設定　
param_grid = [
    {
        'preprocesser__transform__scaler':[StandardScaler(),MinMaxScaler(),RobustScaler()],
        'classifier__C':[0.1,1.0,10.0,100.0],
        'classifier':[LogisticRegression()]
    }
]

grid_search = GridSearchCV(pipeline,param_grid,cv=10,verbose=3,n_jobs=-1)

In [56]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('transform',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['sepal '
                                                                          'length '
                                                                          '(cm)',
                                                                          'sepal '
                                                                          'width '
                                                                          '(cm)',
                                                                          'petal '
                                               

In [57]:
print(grid_search.best_params_)#最適なパイプライン

{'classifier': LogisticRegression(C=100.0), 'classifier__C': 100.0, 'preprocessor__transform__scaler': MinMaxScaler()}


In [58]:
print(grid_search.best_score_)#正答率

0.9857142857142858


In [59]:
#テストデータで精度を検証する
y_test_pred = grid_search.predict(X_test)
print('accuracy :',accuracy_score(y_test,y_test_pred))


accuracy : 0.96


In [60]:
#ランダムサーチ
param_grid = [
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],       
        "classifier__C": [0.1, 1.0, 10.0, 100.0],
        "classifier": [LogisticRegression()]
    },
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__n_estimators": [10, 100, 1000],
        "classifier": [RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]
    },
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__C": [1, 10, 100, 1000],
        "classifier": [SVC(),LinearSVC()]
    }
]

rand_search = RandomizedSearchCV(pipeline, param_grid, cv=10, verbose=3, n_jobs=-1, n_iter=10)

In [ ]:
# ランダムサーチの実行
rand_search.fit(X_train, y_train)

In [ ]:
# ランダムサーチの結果
print(rand_search.best_params_) #最適なパイプライン
print(rand_search.best_score_)  #正答率

In [64]:
#taitanic
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

In [65]:
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3.0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None
1305,3.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C,None,NaN,None
1306,3.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C,None,304.0,None
1307,3.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C,None,NaN,None


In [66]:
y

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: category
Categories (2, object): ['0', '1']

In [67]:
y = y.astype('int')

In [68]:
y

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: int64

In [92]:
#パイプライン構築

##特徴量
numeric_features = ["age", "sibsp", "parch", "fare"] 
categorical_features = ["sex", "pclass"]  
###数値型
numeric_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer()),#欠損値補完
    ('scaler',StandardScaler())#標準化
])

###カテゴリ型
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

###特徴量を変換器にかける
preprocesser = ColumnTransformer(transformers=[
    ('num_transform',numeric_transformer,numeric_features)
])

pipeline = Pipeline(steps=[#変換器パイプラインから予測器へ
    ('preprocesser',preprocesser),
    ('classifier',LogisticRegression())
])




In [93]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [94]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'sibsp', 'parch',
                                                   'fare'])])),
                ('classifier', LogisticRegression())])

In [95]:
y_test_pred1 = pipeline.predict(X_test)
print('accuracy',accuracy_score(y_test,y_test_pred1))
print('f1',f1_score(y_test,y_test_pred1))

accuracy 0.648854961832061
f1 0.28125000000000006


In [ ]:
#optuna
import optuna

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    score = (x - 2) ** 2
    print('x: %1.3f, score: %1.3f' % (x, score))
    return score

study = optuna.create_study()
study.optimize(objective, n_trials=100)

In [85]:
study.best_params

{'x': 2.0200526150770277}

In [96]:
# グリッドサーチの設定
param_grid = [
    {
        "preprocesser__num_transform__imputer__strategy": ["mean", "median"],
        "preprocesser__num_transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],       
        "classifier__C": [0.1, 1.0, 10.0, 100.0],
        "classifier": [LogisticRegression()]
    }
]
grid_search = GridSearchCV(pipeline, param_grid, cv=10, verbose=3,n_jobs=-1)


In [97]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesser',
                                        ColumnTransformer(transformers=[('num_transform',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'sibsp',
                                                                          'parch',
                                                                          'fare'])])),
                                       ('classifier', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()],
                          'classifier__C': [0.1, 1.0, 10.0, 100.0],
                          'preprocesser__num_transform__imputer__strategy': ['mean',
                                                                             'median'],
                          'preprocesser__num_transform__scaler': [StandardScaler(),
                                                                  MinMaxScaler(),
                                                                  RobustScaler()]}],
             verbose=3)

In [99]:
print(grid_search.best_params_) #最適なパイプライン
print(grid_search.best_score_)  #正答率


{'classifier': LogisticRegression(), 'classifier__C': 1.0, 'preprocesser__num_transform__imputer__strategy': 'mean', 'preprocesser__num_transform__scaler': StandardScaler()}
0.6810256410256411


In [101]:
# テストデータで精度検証
y_test_pred = grid_search.predict(X_test)
print('accuracy:',accuracy_score(y_test, y_test_pred))
print('f1:', f1_score(y_test, y_test_pred))

accuracy: 0.648854961832061
f1: 0.28125000000000006


In [104]:
# グリッドサーチの設定その２
param_grid = [
    {
        "preprocesser__num_transform__imputer__strategy": ["mean", "median"],
        "preprocesser__num_transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__C": [0.1, 1.0, 10.0, 100.0],
        "classifier": [LogisticRegression()]
    },
    {
        "preprocesser__num_transform__imputer__strategy": ["mean", "median"],
        "preprocesser__num_transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__n_estimators": [10, 100, 1000],
        "classifier": [RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]
    },
    {
        "preprocesser__num_transform__imputer__strategy": ["mean", "median"],
        "preprocesser__num_transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__C": [1, 10, 100, 1000],
        "classifier": [SVC(),LinearSVC()]
    }
]
grid_search = GridSearchCV(pipeline, param_grid, cv=10, verbose=3,n_jobs=-1)


In [105]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 126 candidates, totalling 1260 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesser',
                                        ColumnTransformer(transformers=[('num_transform',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'sibsp',
                                                                          'parch',
                                                                          'fare'])])),
                                       ('classifier', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()],
                          'classifier__C': [0.1, 1.0, 10.0, 100....
                          'preprocesser__num_transform__imputer__strategy': ['mean',
                                                                             'median'],
                          'preprocesser__num_transform__scaler': [StandardScaler(),
                                                                  MinMaxScaler(),
                                                                  RobustScaler()]},
                         {'classifier': [SVC(C=100), LinearSVC()],
                          'classifier__C': [1, 10, 100, 1000],
                          'preprocesser__num_transform__imputer__strategy': ['mean',
                                                                             'median'],
                          'preprocesser__num_transform__scaler': [StandardScaler(),
                                                                  MinMaxScaler(),
                                                                  RobustScaler()]}],
             verbose=3)

In [107]:
# グリッドサーチの結果
print(grid_search.best_params_) #最適なパイプライン
print(grid_search.best_score_)  #正答率

{'classifier': SVC(C=100), 'classifier__C': 100, 'preprocesser__num_transform__imputer__strategy': 'mean', 'preprocesser__num_transform__scaler': StandardScaler()}
0.723919413919414


In [108]:
# テストデータで精度検証
y_test_pred = grid_search.predict(X_test)
print('accuracy:',accuracy_score(y_test, y_test_pred))
print('f1:', f1_score(y_test, y_test_pred))


accuracy: 0.6984732824427481
f1: 0.5269461077844312


In [110]:
rand_search = RandomizedSearchCV(pipeline, param_grid, cv=10, verbose=3, n_jobs=-1, n_iter=10)

In [111]:
# ランダムサーチの実行
rand_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocesser',
                                              ColumnTransformer(transformers=[('num_transform',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['age',
                                                                                'sibsp',
                                                                                'parch',
                                                                                'fare'])])),
                                             ('classifier',
                                              LogisticRegression())]),
                   n_jobs=-1,
                   param_distributions=[{'classifier': [LogisticRegression()],
                                         'classifier__C': [0....
                                         'preprocesser__num_transform__imputer__strategy': ['mean',
                                                                                            'median'],
                                         'preprocesser__num_transform__scaler': [StandardScaler(),
                                                                                 MinMaxScaler(),
                                                                                 RobustScaler()]},
                                        {'classifier': [SVC(C=100),
                                                        LinearSVC()],
                                         'classifier__C': [1, 10, 100, 1000],
                                         'preprocesser__num_transform__imputer__strategy': ['mean',
                                                                                            'median'],
                                         'preprocesser__num_transform__scaler': [StandardScaler(),
                                                                                 MinMaxScaler(),
                                                                                 RobustScaler()]}],
                   verbose=3)

In [113]:
# ランダムサーチの結果
print(rand_search.best_params_) #最適なパイプライン
print(rand_search.best_score_)  #正答率

{'preprocesser__num_transform__scaler': StandardScaler(), 'preprocesser__num_transform__imputer__strategy': 'median', 'classifier__n_estimators': 100, 'classifier': AdaBoostClassifier(n_estimators=100)}
0.7172802197802198


In [114]:
# テストデータで精度検証
y_test_pred = rand_search.predict(X_test)
print('accuracy:',accuracy_score(y_test, y_test_pred))
print('f1:', f1_score(y_test, y_test_pred))

accuracy: 0.6755725190839694
f1: 0.5251396648044693
